In [1]:
import pandas as pd 
import numpy  as np
from wordfreq import word_frequency ,top_n_list,get_frequency_dict

from symspellpy import SymSpell, Verbosity
from itertools import islice

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from random import random, choice
import random
import time

from tqdm import tqdm  
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True,nb_workers=4)


INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
sym_spell = SymSpell(max_dictionary_edit_distance=5, prefix_length=7)
dictionary_path = 'asr_bangla/data/prothom_alo_word_freq.txt'
sym_spell.load_dictionary(dictionary_path, 0, 1,separator=" ")

True

In [3]:
df_prothom_alo = pd.read_csv('bangla_text_data.csv')

df_prothom_alo= df_prothom_alo.dropna(how='any')

In [4]:
def get_len(sen):
    sen = sen.split()
    l = len(sen)
    return l

In [5]:
df_prothom_alo = df_prothom_alo.sample(300000)

In [6]:
df_prothom_alo['lena'] = df_prothom_alo['sentence'].apply(lambda x: get_len(str(x)) )

In [7]:
df_prothom_alo.lena.value_counts()

6      28800
5      27971
7      27395
4      25624
8      24386
       ...  
123        1
63         1
80         1
58         1
75         1
Name: lena, Length: 73, dtype: int64

In [8]:
df_prothom_alo = df_prothom_alo[(df_prothom_alo.lena >= 3) & (df_prothom_alo.lena <= 15)]

In [9]:
df_prothom_alo = df_prothom_alo.sample(100000)

In [10]:
df_prothom_alo.lena.value_counts()

6     11713
5     11287
7     11226
4     10187
8      9930
9      8800
3      8547
10     7366
11     6179
12     5006
13     4106
14     3129
15     2524
Name: lena, dtype: int64

In [11]:
df_prothom_alo.to_csv('custom_data_for_bert.csv',index=False)

In [12]:
df_prothom_alo = pd.read_csv('custom_data_for_bert.csv')

In [13]:
df_prothom_alo

,sentence,lena
0,দিনটি জমকালোভাবে আমার স্বামী আয়োজন করে,6
1,আরও অর্থ থাকায় বছরের জন্য সঞ্চয়পত্র ক্রয় করেন,8
2,একটি বাংলাদেশ তুমি জাগ্রত জনতার,5
3,সমালোচনার তিরবিদ্ধ টেস্ট অধিনায়ক মুমিনুল হকেরও...,9
4,বিভিন্ন এনজিও চর এলাকায় কাজ করছে,6
...,...,...
99995,তিনি মার কথার দিকে বিশেষ নজর না নিয়ে বেরিয়ে গেলেন,10
99996,গতকাল দুপুরে নামাজের পর সৈয়দুর নিজ ঘরে ঘুমিয়ে ...,9
99997,বেলায়েতরা তিন ভাই,3
99998,আজ বুধবার গণমাধ্যমে পাঠানো বিবৃতিতে বলা হয়,7


In [14]:
character = ['অ','আ','ই','ঈ','উ','ঊ','এ','ঐ','ও','ঔ','ঋ','ঃ','ং','ৎ'
             'ক','খ','গ','ঘ','ঙ','চ', 'ছ','য','জ', 'ঝ','ট','ঠ','ড','র','ড়','ঢ়',
             'ণ','ত','থ','দ','ধ','ন','প','ফ','ব','ভ','ম','স','শ','ষ','য়','হ']

In [15]:
JuktakkhorList = [' ক্ট ' , ' ক্ক ' , ' ক্ত ' , ' ক্য ' , ' ক্র ' , ' ক্ল ' , ' ক্ষ ' , ' ক্ষ্ণ ' , ' ক্ষ্ম ' , ' ক্ষ্য ' , ' ক্স ' , ' খ্র ' , ' গ্ধ ' , ' গ্ধ্য ' , ' গ্ন ' , ' গ্ন্য ' , ' গ্ব ' , ' গ্র ' , ' গ্র্য ' , ' গ্ল ' , ' ঘ্ন ' , ' ঘ্র ' , ' ঙ্ক্য ' , ' ঙ্গ্য ' , ' চ্চ ' , ' চ্ছ্ব ' , ' চ্য ' , ' জ্জ ' , ' জ্জ্ব ' , ' ট্ট ' , ' জ্ব ' , ' জ্য ' , ' জ্র ' , ' ট্য ' , ' ট্র ' , ' ড্ড ' , ' ড্র ' , ' ণ্ট ' , ' ণ্ঠ ' , ' ণ্ড ' , ' ণ্ণ ' , ' ণ্য ' , ' ৎক ' , ' ৎখ ' , ' ত্ত ' , ' ত্ত্ব ' , ' ত্ত্য ' , ' ত্ন ' , ' ৎপ ' , ' ত্ব ' , ' ত্ম ' , ' ত্ম্য ' , ' ত্য ' , ' ত্র ' , ' ত্র্য ' , ' ৎস ' , ' দ্ঘ ' , ' দ্দ ' , ' দ্ধ ' , ' দ্ব ' , ' দ্ভ ' , ' দ্ভ্র ' , ' দ্ম ' , ' দ্য ' , ' দ্র ' , ' দ্র্য ' , ' ধ্ব ' , ' ধ্য ' , ' ধ্র ' , ' ন্ট ' , ' ন্ট্র ' , ' ন্ঠ ' , ' ন্ড ' , ' ন্ড্র ' , ' ন্ত ' , ' ন্ত্ব ' , ' ন্ত্য ' , ' ন্ত্র ' , ' ন্ত্র্য ' , ' ন্থ ' , ' ন্দ ' , ' ন্দ্ব ' , ' ন্দ্র ' , ' ন্ধ ' , ' ন্ন ' , ' ন্য ' , ' প্ট ' , ' প্ত ' , ' প্ন ' , ' প্প ' , ' প্য ' , ' ব্দ ' , ' ব্ধ ' , ' ব্ব ' , ' ব্র ' , ' ভ্য ' , ' ভ্র ' , ' ম্প্র ' , ' ম্ব ' , ' ম্ম ' , ' ম্য ' , ' ম্র ' , ' য্য ' , ' র্ক ' , ' র্গ্য ' , ' র্ঘ্য ' , ' র্জ্য ' , ' র্থ্য ' , ' র্ব্য ' , ' র্খ ' , ' র্গ ' , ' র্ঘ ' , ' র্চ ' , ' র্ছ ' , ' র্জ ' , ' র্ঝ ' , ' র্ট ' , ' র্ড ' , ' র্ণ ' , ' র্ত ' , ' র্থ ' , ' র্দ ' , ' র্দ্ব ' , ' র্দ্র ' , ' র্ধ ' , ' র্ধ্ব ' , ' ল্ট ' , ' ল্ড ' , ' ল্প ' , ' ল্ম ' , ' ল্য ' , ' ল্ল ' , ' শ্ছ ' , ' শ্ন ' , ' শ্ব ' , ' শ্র ' , ' শ্ল ' , ' ষ্ক ' , ' ষ্ক্র ' , ' ষ্ট ' , ' ষ্ট্য ' , ' ষ্ট্র ' , ' ষ্ঠ ' , ' ষ্প ' , ' স্ট ' , ' স্ট্র ' , ' স্ত্র ' , ' স্ত ' , ' স্থ্য ' , ' স্য ' , ' স্র ' , ' স্ল ' , ' হ্র ' , ' হ্ল ' , ' গ্ম ' , ' ম্ভ ' , ' স্ম ' , ' ঙ্ঘ ' , ' ঙ্ক্ষ ' , ' ঙ্খ ' , ' ঙ্ক ' , ' ঙ্‌ক্ত ' , ' ঞ্চ ' , ' ঞ্ছ ' , ' ঞ্জ ' , ' হ্ন ' , ' ঙ্গ ' , ' জ্ঞ ' , ' গ্য ' , ' চ্ছ '] 

In [16]:
ReplaceDict = {
      'ক' : ['ল','য'],
      'খ' : ['কগ','কজ','লহ','ঝ'],
      'গ' : ['ফ','হ'],
      'ঘ' : ['ফগ','হজ'],
      'ঙ' : ['ব','ম'],
      'চ' : ['ভ','চজ','চগ','ভহ'],
      'ছ' : ['ভ','চজ','চগ','ভহ'],
      'জ' : ['ক','হ'],
      'ঝ' : ['কজ','হগ'],
      'ট' : ['র'],
      'ঠ' : ['তজ','তগ','রহ'],
      'ড' : ['স','ফ'],
      'ঢ' : ['দজ','দ্গ','শ','ফহ'],
      'ণ' : ['ব','ম'],
      'ত' : ['র'],
      'থ' : ['তজ', 'তগ', 'রহ'],
      'দ' : ['স', 'ফ'],
      'ধ' : ['দজ', 'দ্গ', 'শ' ,'ফহ'],
      'ন' : ['ব','ম'],
      'প' : ['ও',' ো'],
      'ফ' : ['দ','গ'],
      'ব' : ['ভ','ন'],
      'ভ' : ['‍ব','চ'],
      'ম' : ['ন'],
      'য' : ['হ','ক'],
      'র' : ['এ',' ে','ত'],
      'ল' : ['ক'],
      'শ' : ['সজ','সগ','আহ','ঢ'],
      'ষ' : ['সজ','সগ','আহ','ঢ'],
      'স' : ['আ',' া','দ'],
      'হ' : ['গ','য'],
      'য়' : ['ত','উ','ু'],
      'ড়' : ['এ',' ে','ত'],
      'ঢ়' : ['এ',' ে','ত'],
      'ৎ' : ['র'],
      'ং' : ['ব','ম'],
      'ঃ' : ['গ','য'],
      'অ' : ['প',' ি','ই'],
      'আ' : ['স'],
      'ই' : ['উ','অ'],
      'ঈ' : ['উ','অ'],
      'উ' : ['ই',' ি'],
      'ঊ' : ['ই',' ি'],
      'ঋ' : [''],
      'এ' : ['ও','র'],
      'ঐ' : [''],
      'ও' : ['প','ই'],
      'ঔ' : [''],
      'া' : ['স'],
      'ি' : ['উ','অ'],
      'ো' : ['প',' ি'],
      'ৌ' : [''],
      'ে' : ['ো','র'],
      'ৈ' : ['']
    }

In [17]:
SameClusterDict = {
    'অ' : ['ও','আ','অ্যা','া','ওঁ','ওঃ'],
    'আ' : ['আ','অ্যাঁ','আঃ','ওয়া','এ'],
    'ই':['ই','ঈ','এ','ে','য়'],
    'ঈ':['ই','ঈ','এ','ে','য়'],
    'উ' : ['উ''উঁ','ঊ','ু'],
    'ঊ' : ['উ''উঁ','ঊ','ু'],
    'ঋ' : ['রি','হ্রী','ড়ি','ঢ়ি','ব্রই','ৃ','হ্র'],
    'এ' : ['এ','অ্যা','আঃ'],
    'ঐ': ['অই','ঈ','ওই' , 'ই', 'অ্যাই'],
    'ও': ['ও','আ','অ্যা','া','ওঁ','ওঃ'],
    'ঔ': [ 'ও','আ','অউ','অ্যা','া','ওঁ','ওঃ'],
    'ক': ['ক','খ','গ', 'ও','আ','অ্যা','া','ওঁ','ওঃ','কোঁ',' কো','খ','খো','খও','ক্ষও','কহোঁ','কহও','গোঁ','গয়' ],
    'খ': ['ক','খ','গ', 'ও','আ','অ্যা','া','ওঁ','ওঃ','কোঁ',' কো','খ','খো','খও','ক্ষও','কহোঁ','কহও','গোঁ','গয়' ],
    'গ': ['ক','খ','গ', 'ও','আ','অ্যা','া','ওঁ','ওঃ','কোঁ',' কো','খ','খো','খও','ক্ষও','কহোঁ','কহও','গোঁ','গয়' ],
    'ঘ ': ['খ','খো','খও','ক্ষও','কহোঁ','কহও','গোঁ','গয়' ,'ঘো ','ঘও'],
    'ঙ': ['◌ং','ও','আ','অ্যা','া','ওঁ','ওঃ','অং'],
    'চ': ['চ','ও','আ','অ্যা','া','ওঁ','ওঃ','চোঁ','চও','ছোঁ','ছো','ছও','ছ'],
    'ছ': ['চ','ও','আ','অ্যা','া','ওঁ','ওঃ','চোঁ','চও','ছোঁ','ছো','ছও','ছ'],
    'জ': ['জ','য','জো','যো','ঝ','ঝও'],
    'ঝ': ['জ','য','জো','যো','ঝ','ঝও'],
    'ঞ': ['ন','ঙ','ণ','ঞ','ই','ঈ','এ','ে','য়','নও' ],
    'ট' : ['ত','তঁ','তোঁ','টয়','তো','ট'],
    'ঠ' : ['থ','ত','তঁ','তোঁ','টয়','তো','ট'],
    'ড' : ['দ','দো','দ্য','ডঃ' ,'দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'ঢ': ['দ','দো','দ্য','ডঃ','দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'ণ': ['ন','ঙ','ণ','ঞ','ই','ঈ','এ','ে','য়','নও' ],
    'ত' : ['দ','দো','দ্য','ডঃ' ,'দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'থ': ['দ','দো','দ্য','ডঃ' ,'দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'দ' : ['দ','দো','দ্য','ডঃ' ,'দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'ধ' : ['দ','দো','দ্য','ডঃ' ,'দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'ন': ['ন','ঙ','ণ','ঞ','ই','ঈ','এ','ে','য়','নও' ],
    'প' : ['প','ফ','ব','ভ','ম','ব্য', 'ভঁ', 'বও' ,'বো','বোঁ','ব্যয়','পঅ' ,'পো' ,'পয়' ,'পোঁ','ভঁ' ,'ভঅ' ,'ভয়','ভোঁ','ময়' ,'মোঃ'],
    'ফ' : ['প','ফ','ব','ভ','ম','ব্য', 'ভঁ', 'বও' ,'বো','বোঁ','ব্যয়','পঅ' ,'পো' ,'পয়' ,'পোঁ','ভঁ' ,'ভঅ' ,'ভয়','ভোঁ','ময়' ,'মোঃ'],
    'ব' : ['প','ফ','ব','ভ','ম','ব্য', 'ভঁ', 'বও' ,'বো','বোঁ','ব্যয়','পঅ' ,'পো' ,'পয়' ,'পোঁ','ভঁ' ,'ভঅ' ,'ভয়','ভোঁ','ময়' ,'মোঃ'],
    'ভ' : ['প','ফ','ব','ভ','ম','ব্য', 'ভঁ', 'বও' ,'বো','বোঁ','ব্যয়','পঅ' ,'পো' ,'পয়' ,'পোঁ','ভঁ' ,'ভঅ' ,'ভয়','ভোঁ','ময়' ,'মোঃ'],
    'ম' : ['প','ফ','ব','ভ','ম','ব্য', 'ভঁ', 'বও' ,'বো','বোঁ','ব্যয়','পঅ' ,'পো' ,'পয়' ,'পোঁ','ভঁ' ,'ভঅ' ,'ভয়','ভোঁ','ময়' ,'মোঃ'],
    'য' : ['প','ফ','ব','ভ','ম','ব্য', 'ভঁ', 'বও' ,'বো','বোঁ','ব্যয়','পঅ' ,'পো' ,'পয়' ,'পোঁ','ভঁ' ,'ভঅ' ,'ভয়','ভোঁ','ময়' ,'মোঃ'],
    'র' : ['র','ড়','ঢ়','রও','রোঁ','রো','রয়','ঋ', 'হর'  ],
    'ল' : ['ল','লয়','লো'],
    'শ' : ['শ','ষ','স','ও','আ','অ্যা','া','ওঁ','ওঃ','শো','সঅ','শয়'],
    'ষ' : ['শ','ষ','স','ও','আ','অ্যা','া','ওঁ','ওঃ','শো','সঅ','শয়'],
    'স' : ['শ','ষ','স','ও','আ','অ্যা','া','ওঁ','ওঃ','শো','সঅ','শয়'],
    '‍হ': ['হ','ও','আ','অ্যা','া','ওঁ','ওঃ'],
    'ড়' : ['র','ড়','ঢ়','রও','রোঁ','রো','রয়','ঋ', 'হর'  ],
    '‍ঢ়': ['র','ড়','ঢ়','রও','রোঁ','রো','রয়','ঋ', 'হর'  ],
    '‍য় ': ['ই','ঈ','এ','ে','য়'],

    'ৎ' : ['দ','দো','দ্য','ডঃ' ,'দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'ং' : ['◌ং','ও','আ','অ্যা','া','ওঁ','ওঃ','অং'],
    'ঃ': ['হ','ও','আ','অ্যা','া','ওঁ','ওঃ'],
    '‍ঁ' : [''],

    '‍ি': ['ই','ঈ','এ','ে','য়'],
    'ী' : ['ই','ঈ','এ','ে','য়'],
    'ে':['এ','অ্যা','আঃ'],
    'ৈ'  :['এ','অ্যা','আঃ'],
    'ো' : ['ও','আ','অ্যা','া','ওঁ','ওঃ'],
    'ৌ':['ও','আ','অ্যা','া','ওঁ','ওঃ'],
    'ৃ' : ['রি','হ্রী','ড়ি','ঢ়ি','ব্রই','ৃ','হ্র'],
     '◌ূ'  : ['উ''উঁ','ঊ','ু'],
     ' ু'  : ['উ''উঁ','ঊ','ু']
}

In [37]:
def apply_error(string,error):
    random.seed(time.time()+200)
    errors = ['Cluster','Replace','Juktakkhor','Character','Deletion','Noerror']
    if error == "Cluster":
        if string in list(SameClusterDict.keys()):
            string = random.choice(SameClusterDict[string])
    if error == "Replace":
        if string in list(ReplaceDict.keys()):
            string = random.choice(ReplaceDict[string])
    if error == "Juktakkhor":
        string = random.choice(JuktakkhorList)
    if error == "Character":
        string = random.choice(character)
    if error == "Deletion":
        string = ""
    if error == "insertion":
        temp_error = random.choice(errors)
        string = string + apply_error(string,temp_error)
    if error == "Noerror":
        pass
    return string

In [63]:
def error_generator(word):
    errors = ['Cluster','Replace','Juktakkhor','Character','Deletion','Insertion','Noerror']
    string = []
    string[:0] = word
    random.seed(time.time())
    noise_ratio = random.randint(0,int(len(word)*0.40))
    print(noise_ratio)
    for n in range(noise_ratio):
        for i in range(len(string)):
            random.seed(time.time()+2)
            error_t = random.random()
            if error_t >=0.5:
                random.seed(time.time()+100)
                error = random.choice(errors)
                string[i] = apply_error(string[i],error)
    word = "".join(string)
    word = word.replace(" ", "")
    return word

In [64]:
word = 'অজপারাগায়ে'

In [65]:
error_generator(word)

4


'ত্রএফসরপ্টইওল্লট'

In [33]:
sentence = 
df_prothom_alo.sentence.tolist()

In [34]:
df_custom_error_dataset = pd.DataFrame()
for sen in tqdm(sentence):
    sent = sen
    sen = sen.split()
    sent = sent.split()
    random.seed(time.time()+356)
    random_index = random.randint(0,len(sen)-1)
    sen[random_index] = '[MASK]'
    error_word = error_generator(sent[random_index])
    temp_dict = {'sentence':" ".join(sent),'masked_sen':" ".join(sen),'error_word':error_word }
    df_custom_error_dataset = df_custom_error_dataset.append(temp_dict,ignore_index=True)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [24:06<00:00, 69.15it/s]


In [35]:
df_custom_error_dataset.sample(5)

,sentence,masked_sen,error_word
87101,সেখানে বেলা তিনটায় তাঁকে মায়ের কবরে চিরনিদ্রায়...,সেখানে বেলা তিনটায় তাঁকে মায়ের কবরে [MASK] শায়...,চগযরোৎপদঙ্‌ক্তন্দ্বদ্ভ্র
86466,এরপর নিজেদের দৈনন্দিন হিসাবনিকাশ শেষ করে আটটার...,এরপর নিজেদের দৈনন্দিন হিসাবনিকাশ শেষ করে আটটার...,বন্ধ
77153,স্থায়ী ঠিকানায় জায়গাজমি না থাকলে জটিলতাগুলো কে...,স্থায়ী ঠিকানায় জায়গাজমি না [MASK] জটিলতাগুলো ক...,তকথো
74888,আশ্রয়শিবিরের সবকিছু দেখভাল করে দুর্যোগ ব্যবস্থ...,আশ্রয়শিবিরের সবকিছু দেখভাল করে দুর্যোগ ব্যবস্থ...,পোনঘ্নদআব্রর্টঙঃও
68280,আমার যতটুকু মনে পড়ে,আমার যতটুকু [MASK] পড়ে,স্ট্রআে


In [36]:
df_custom_error_dataset.to_csv('custom_error_dataset.csv',index=False)